               SCRAPING RIGHTMOVE

In [ ]:
# This is the code to first scrape the urls
# importing our libraries

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random
import json

#creating a dictionary of boroughs and their rightmove codes in London
BOROUGHS = {
   "City of London": "5E61224",
    "Barking and Dagenham": "5E61400",
#     "Barnet": "5E93929",
#     "Bexley": "5E93932",
#     "Brent": "5E93935",
#     "Bromley": "5E93938",
#     "Camden": "5E93941",
#     "Croydon": "5E93944",
#     "Ealing": "5E93947",
#     "Enfield": "5E93950",
#     "Greenwich": "5E61226",
#     "Hackney": "5E93953",
#     "Hammersmith and Fulham": "5E61407",
#     "Haringey": "5E61227",
#     "Harrow": "5E93956",
#     "Havering": "5E61228",
#     "Hillingdon": "5E93959",
#     "Hounslow": "5E93962",
#     "Islington": "5E93965",
#     "Kensington and Chelsea": "5E61229",
#     "Kingston upon Thames": "5E93968",
#     "Lambeth": "5E93971",
#     "Lewisham": "5E61413",
#     "Merton": "5E61414",
#     "Newham": "5E61231",
#     "Redbridge": "5E61537",
#     "Richmond upon Thames": "5E61415",
#     "Southwark": "5E61518",
#     "Sutton": "5E93974",
#     "Tower Hamlets": "5E61417",
#     "Waltham Forest": "5E61232",
#     "Wandsworth": "5E93977",
#     "Westminster": "5E93980",
}
Links = []

for borough in list(BOROUGHS.values()):

        # initialise index, this tracks the page number we are on. every additional page adds 24 to the index
        index = 0

        key = [key for key, value in BOROUGHS.items() if value == borough]
       
        for pages in range(41):

            # define our user headers
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
            }

            # the website changes if the you are on page 1 as compared to other pages
            if index == 0:
                url = f"https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%{borough}&sortType=6&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

            elif index != 0:
                url = f"https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%{borough}&sortType=6&index={index}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

            # request our webpage
            response = requests.get(url, headers=headers)

            soup = BeautifulSoup(response.text, "html.parser")

            # This gets the list of apartments
            listings = soup.find_all("div", class_="l-searchResult is-list")

            for i in range(len(listings)):

                # tracks which apartment we are on in the page
                listing_no = listings[i]

                # append link
                listing_info = listing_no.find("a", class_="propertyCard-link")
                link = "https://www.rightmove.co.uk" + listing_info.attrs["href"]
                Links.append(link)

            # code to ensure that we do not overwhelm the website
            time.sleep(random.randint(1, 3))
             
            # Code to count how many listings we have scrapped already.
            index = index + 24

            if index >= number_of_listings:
                break

    # convert data to dataframe
data = {
        "Links": Links,
       }


           CONVERTING THE LINKS TO A DATAFRAME AND SAVING A COPY AS A CSV FILE

In [ ]:
#Converting the data to a dataframe and links_list
df = pd.DataFrame.from_dict(data)

#removing duplicates
df_rightmove_links= df.drop_duplicates()

#saving the links as a csv file
df_rightmove_links.to_csv(r"results/RL.csv", encoding="utf-8", header="true", index = False)

#cleaning up the links
df_rightmove_links['Links'] = df_rightmove_links['Links'].str.replace(r'\?channel=RES_BUY', '')


# Convert the link dataframe back to links_list
Links_list = df_rightmove_links['Links'].tolist()

                                  SCRAPING THE URLS

In [ ]:
pip install httpx parsel jmespath httpx[http2] scrapfly-sdk scrapy

In [ ]:
import asyncio
import json
from typing import List
from httpx import AsyncClient, Response
from parsel import Selector
from pathlib import Path
import nest_asyncio
import jmespath

nest_asyncio.apply()

from typing import TypedDict

# 1. establish HTTP client with browser-like headers to avoid being blocked
client = AsyncClient(
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
    },
    follow_redirects=True,
    http2=True,  # enable http2 to reduce block chance
    timeout=30,
)

class PropertyResult(TypedDict):
    """this is what our result dataset will look like"""
    id: str
    status: bool
    title: str
    description: str
    property_type: str
    bedrooms: int
    bathrooms: int
    price: str
    price_sqft: str
    address: dict
    features: list
    latitude: float
    longitude: float
    photos: list
    floorplans: list
    Listinghistory: str
    Agent_Name:str
    Agent_No: str
    Agent_Address: str
    nearest_airports: list
    nearest_stations: list
    Tenure_type: bool
    Lease_year_left:str
    CouncilTax_Status:str
    annualGroundRent:str
    annualServiceCharge:str
    councilTaxBand:str
    domesticRates:str
def parse_property(data) -> PropertyResult:
    """parse rightmove cache data for property information"""
    # here we define field name to JMESPath mapping
    parse_map = {

           "id": "id",
           "Status": "status.published",
            "title": "text.pageTitle",
            "description": "text.description",
            "property_type": "propertySubType",
            "bedrooms": "bedrooms",
            "bathrooms": "bathrooms",
            "price": "prices.primaryPrice",
            "price_sqft": "prices.pricePerSqFt",
            "address": "address",
            "features": "keyFeatures",
            "latitude": "location.latitude",
            "longitude": "location.longitude",
            "photos": "images[:1].{url: url, caption: caption}",
            "floorplans": "floorplans[:1].{url: url, caption: caption}",  
            "Listinghistory": "listingHistory.listingUpdateReason",
            'Agent_Name': 'customer.companyName',
            'Agent_No': "contactInfo.telephoneNumbers.localNumber",
            'Agent_Address': 'customer.displayAddress',            
            "nearest_airports": "nearestAirports[*].{name: name, distance: distance}",
             "nearest_stations": "nearestStations[*].{name: name, distance: distance}",
            'Tenure_type': 'tenure.tenureType',
             'Lease_year_left': 'tenure. yearsRemainingOnLease',
             'CouncilTax_Status': 'livingCosts. councilTaxExempt',
            'annualGroundRent': 'livingCosts.annualGroundRent',
             "annualServiceCharge": 'livingCosts.annualServiceCharge',
             "councilTaxBand":  "livingCosts.councilTaxBand",
              "domesticRates": "livingCosts.domesticRates",
    

    }
    results = {}
    for key, path in parse_map.items():
        value = jmespath.search(path, data)
        results[key] = value
    return results

def extract_property(response: Response) -> dict:
    """extract property data from rightmove PAGE_MODEL javascript variable"""
    selector = Selector(response.text)
    data = selector.xpath("//script[contains(.,'PAGE_MODEL = ')]/text()").get()
    if not data:
        print(f"page {response.url} is not a property listing page")
        return
    data = data.split("PAGE_MODEL = ", 1)[1].strip()
    data = json.loads(data)
    return data["propertyData"]

async def scrape_properties(urls: List[str], delay: float = 0.0) -> List[dict]:
    """Scrape Rightmove property listings for property data"""
    to_scrape = [client.get(url) for url in urls]
    properties = []
    for response in asyncio.as_completed(to_scrape):
        response = await response
        properties.append(parse_property(extract_property(response)))
        await asyncio.sleep(delay)  # Introduce a delay between requests
    return properties

async def run():
    """example run calls for our scraper functions"""
    out_path = Path.cwd() / "results"
    out_path.mkdir(exist_ok=True)

   
    data = await scrape_properties(Links_list)
    out_path.joinpath("rm_properties_all.json").write_text(json.dumps(data, indent=2))


if __name__ == "__main__":
    asyncio.run(run())


In [350]:
import pandas as pd

# Read the JSON file
with open('results/rm_properties_all.json', 'r') as file:
    json_data = json.load(file)

# Convert JSON data to DataFrame
df_rm = pd.DataFrame(json_data)

# save a copy of dataframe as csv (commented out in order not to overwrite what has already been saved)
#df_rm.to_csv('results/rm_properties_all.csv', index=False)


                          SCRAPING ZOOPLA

In [ ]:
#This is the code to scrape out all the Urls and save them as csv file in a directory called results
import time
import pandas as pd
import reimport json
from urllib.parse import urljoin
from scrapfly import ScrapflyClient, ScrapeConfig, ScrapeApiResponse
BOROUGHS = {
     "City of London": "5E61224",
    "Barking and Dagenham": "5E61400",
    #"Barnet": "5E93929",
#     "Bexley": "5E93932",
    # "Brent": "5E93935",
    # "Bromley": "5E93938",
    # "Camden": "5E93941",
    # "Croydon": "5E93944",
    # "Ealing": "5E93947",
    # "Enfield": "5E93950",
    # "Greenwich": "5E61226",
    # "Hackney": "5E93953",
    # "Hammersmith and Fulham": "5E61407",
    # "Haringey": "5E61227",
    # "Harrow": "5E93956",
    # "Havering": "5E61228",
    # "Hillingdon": "5E93959",
    # "Hounslow": "5E93962",
    # "Islington": "5E93965",
    # "Kensington and Chelsea": "5E61229",
    # "Kingston upon Thames": "5E93968",
    # "Lambeth": "5E93971",
    # "Lewisham": "5E61413",
    # "Merton": "5E61414",
    # "Newham": "5E61231",
    # "Redbridge": "5E61537",
    # "Richmond upon Thames": "5E61415",
    # "Southwark": "5E61518",
    # "Sutton": "5E93974",
    # "Tower Hamlets": "5E61417",
    # "Waltham Forest": "5E61232",
    # "Wandsworth": "5E93977",
    # "Westminster": "5E93980",
}

Urls = []
locations = [key.replace(" ", "-").lower() for key in BOROUGHS.keys()]
base_url = "https://www.zoopla.co.uk/for-sale/property/{}/?pn="

for location in locations:
    for pn in range(1, 41):  # Scrape pages 1 and 2 
        url = base_url.format(location) + str(pn)
     
        client = ScrapflyClient(key="scp-live-595ffb14156649c2b839689a8987c9e1")
        result: ScrapeApiResponse = client.scrape(ScrapeConfig(
            url=url,
            # we can select specific country:
            country="GB",
            # proxy type:
            proxy_pool="public_residential_pool",
            # we can also enable headless browser powered js rendering
            render_js=True,
        ))
        
        for card in result.soup.select('[data-testid="regular-listings"] [id^="listing"]'):
            link = "https://www.zoopla.co.uk" + card.a.get("href")
            Urls.append(link)
data = {
        "urls": Urls,
       }
df_zoopla_links = pd.DataFrame.from_dict(data)

In [ ]:
#df_zoopla_links.to_csv(r"results/ZL.csv", encoding="utf-8", header="true", index = False)

In [ ]:
import csv

file_path = 'results/ZL.csv'
urls = []

with open(file_path, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        urls.append(row[0])

In [ ]:
pip install scrapfly[scrapy]

In [ ]:
#import asyncio
#from typing import List, Literal, Optional, TypedDict,Dict,Any,Callable
import json
import math


import nest_asyncio
import jmespath
import pandas as pd
from httpx import AsyncClient, Response
from parsel import Selector

nest_asyncio.apply()


from typing import TypedDict

import jmespath
from scrapfly import ScrapeConfig, ScrapeApiResponse, ScrapflyClient

#scrapfly = ScrapflyClient(key="scp-live-595ffb14156649c2b839689a8987c9e1", max_concurrency=2)


class PropertyResult(TypedDict):
    listing_id: str
    title: str
    description: str
    url: str
    price: str
    type: str
    date: str
    category: str
    section: str
    features: List[str]
    floor_plan: Dict[str, str]
    nearby: List[dict]
    coordinates: Dict[str, float]
    photos: List[dict]
    details: Dict[str, str]
    agency: Dict[str, str]


def parse_property(result: Callable[..., Any]) -> Optional[PropertyResult]:
    data = extract_next_data(result)
    if not data:
        return None
    result = jmespath.search(
        """root.{
        id: listingId,
        title: title,
        description: detailedDescription,
        url: listingUris.detail,
        price: pricing.label,
        type: propertyType,
        date: publishedOn,
        category: category,
        section: section,
        features: features.bullets,
        floor_plan: floorPlan.image.{filename:filename, caption: caption}, 
        nearby: pointsOfInterest[].{title: title, distance: distanceMiles},
        coordinates: location.coordinates.{lat:latitude, lng: longitude},
        photos: propertyImage[].{filename: filename, caption: caption},
        details: analyticsTaxonomy,
        agency: branch
    }""", {"root": data["listingDetails"]})
    return result

def extract_next_data(result: ScrapeApiResponse) -> dict:
    """find hidden next.js data in page scrape result"""
    data = result.selector.css("script#__NEXT_DATA__::text").get()
    if not data:
        print(f"page {result.context['url']} is not a property listing page")
        return
    data = json.loads(data)
    return data["props"]["pageProps"]


async def scrape_properties(urls: List[str]) -> List[PropertyResult]:
    """scrape Zoopla property pages and parse results"""
    to_scrape = [ScrapeConfig(url=url, asp=True, country="GB") for url in urls]
    properties = []
    async for result in scrapfly.concurrent_scrape(to_scrape):
        parsed_result = parse_property(result)
        properties.append(parsed_result)
    return properties


if __name__ == "__main__":
        
    zp_properties_all = asyncio.run(scrape_properties(urls))

    df = pd.DataFrame(zp_properties_all)

     # Save as CSV(commented out in order not to overwrite what has already been saved)
    #df.to_csv("results/zp_properties_all.csv", index=False)
    
    # Save as JSON
    #df.to_json("results/zp_properties_all.json", orient="records")


                   WORKING ON SCRAPED RIGHTMOVE DATA

In [337]:

#READING THE SCRAPED CSV FILE INTO A DATAFRAME CALLED df_rm
df_rm=pd.read_csv('results/rm_properties_all.csv')

from bs4 import BeautifulSoup
#replacing boolean to string in status and CouncilTax_Status column
df_rm['Status'] = df_rm['Status'].replace({True: 'Available', False: 'Unavailable'})
df_rm['CouncilTax_Status'] = df_rm['CouncilTax_Status'].replace({True: 'Not Exempted', False: 'Exempted'})

#cleaning the agen address column
df_rm['Agent_Address'] = df_rm['Agent_Address'].str.replace('\r\n', ' ')

#cleaning up the nearest station
df_rm['nearest_stations'] = df_rm['nearest_stations'].apply(lambda x: ast.literal_eval(x))
df_rm['nearest_stations'] = df_rm['nearest_stations'].apply(lambda x: [f"{item['name']}:{item['distance']}" for item in x])
df_rm['nearest_stations'] = df_rm['nearest_stations'].apply(lambda x: ', '.join(x))

#cleaning and refining the description column
df_rm['description'] = df_rm['description'].str.replace('<strong>', '').str.replace('</strong>', '').str.replace('<br>', '').str.replace('<br/><br/>', '')

df_rm['address'] = df_rm['address'].apply(lambda x: ast.literal_eval(x))
df_rm['address']=df_rm['address'].apply(lambda x: x['displayAddress'])
df_rm['address'] = df_rm['address'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

#cleaning and refining photos and floorplans column

#cleaning and refining photos and floorplans column
df_rm['photos'] = df_rm['photos'].apply(lambda x: ast.literal_eval(x))
df_rm['photos'] = df_rm['photos'].apply(lambda x: [item['url'] for item in x])

df_rm['floorplans'] = df_rm['floorplans'].apply(lambda x: ast.literal_eval(x))
df_rm['floorplans'] = df_rm['floorplans'].apply(lambda x: [item['url'] for item in x])

df_rm['nearest_airports'] = df_rm['nearest_airports'].str.replace('[', '').str.replace(']', '')

df_rm['features'] =df_rm['features'].str.replace('[', '').str.replace(']', '')

df_rm['description'] = df_rm['description'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())


#Prefixing all the columns of df_rm with RM
df_rm = df_rm.add_prefix('RM_')

#Creating a unique column called Coordinate by combining the latititude and longitude with semicolon
df_rm['Coordinate'] = df_rm['RM_latitude'].astype(str) + ': ' + df_rm['RM_longitude'].astype(str)

#making the coordinate column the first column
df_rm = df_rm[['Coordinate'] + df_rm.columns[:-1].tolist()]

#FINAL RIGHTMOVE DATAFRAME
df_rm


<ipython-input-337-1e790aebb24a>:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_rm['nearest_airports'] = df_rm['nearest_airports'].str.replace('[', '').str.replace(']', '')
<ipython-input-337-1e790aebb24a>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_rm['features'] =df_rm['features'].str.replace('[', '').str.replace(']', '')


,Coordinate,RM_id,RM_Status,RM_title,RM_description,RM_property_type,RM_bedrooms,RM_bathrooms,RM_price,RM_price_sqft,...,RM_Agent_Address,RM_nearest_airports,RM_nearest_stations,RM_Tenure_type,RM_Lease_year_left,RM_CouncilTax_Status,RM_annualGroundRent,RM_annualServiceCharge,RM_councilTaxBand,RM_domesticRates
0,51.5166: -0.080557,74504326,Available,1 bedroom apartment for sale in One Bishopsgat...,"This 21st floor, one bedroom apartment is well...",Apartment,1.0,1.0,"£1,325,000",£2086.61 per sq. ft.,...,33 Margaret Street London W1G 0JD,,"Liverpool Street Station:0.12651093205178365, ...",LEASEHOLD,995.0,Exempted,0.0,0.0,H,NaN
1,51.514636: -0.109152,133932536,Available,"Studio apartment for sale in Crane Court, Lond...",Recently refurbished studio in Central London....,Apartment,NaN,1.0,"£450,000",NaN,...,"122-124 St Johns Street, London, EC1V 4JS",,"City Thameslink Station:0.24488265468523587, C...",SHARE_OF_FREEHOLD,992.0,Exempted,0.0,3861.7,C,NaN
2,51.519342: -0.089223,128173136,Available,"Studio flat for sale in The Heron, Moorgate, L...","Set on the 14th floor, this immaculate studio ...",Studio,NaN,NaN,"£575,000",£1399.03 per sq. ft.,...,"50 - 54 Clerkenwell Road, London, EC1M 5PS",,"Moorgate Station:0.07283115172116814, Barbican...",LEASEHOLD,NaN,Exempted,NaN,NaN,NaN,NaN
3,51.520576: -0.09282,134347634,Available,"4 bedroom flat for sale in Barbican, Cromwell ...",This seventh floor flat has been updated with ...,Flat,4.0,2.0,"£1,650,000",NaN,...,"87 Long Lane, London, EC1A 9ET",,"Barbican Station:0.21907021331390783, Moorgate...",NaN,NaN,Exempted,NaN,NaN,NaN,NaN
4,51.530162: 0.13347,131240564,Available,"3 bedroom house for sale in Shaw Avenue, Barki...",Maybury Estates are delighted to offer for sal...,House,3.0,1.0,"£425,000",NaN,...,"Suite 11a, Stuart House 56 Longbridge Road, Ba...",,"Dagenham Dock Station:0.6219689510363213, Beco...",FREEHOLD,NaN,Exempted,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,51.61835: -0.176293,132642500,Available,6 bedroom semi-detached house for sale in Rave...,Situated in this popular tree lined residentia...,Semi-Detached,6.0,2.0,"£1,250,000",NaN,...,"365 Ballards Lane, London, N12 8LL",,"Woodside Park Station:0.3915216797005928, West...",FREEHOLD,NaN,Exempted,NaN,NaN,NaN,NaN
1781,51.587894: -0.209775,134184077,Available,5 bedroom semi-detached house for sale in Sout...,Talbots are pleased to be able to offer this c...,Semi-Detached,5.0,2.0,"£1,250,000",NaN,...,"67 Brent Street, London, NW4 2EA",,"Brent Cross Station:0.7895905731847184, Hendon...",FREEHOLD,NaN,Exempted,NaN,NaN,E,NaN
1782,51.598311: -0.1738,134997035,Available,4 bedroom semi-detached house for sale in Sylv...,A spacious semi-detached family home with thre...,Semi-Detached,4.0,1.0,"£860,000",NaN,...,"11 Fortis Green, East Finchley, London, N2 9JR",,"Finchley Central Station:0.8269625104446913, E...",FREEHOLD,0.0,Exempted,0.0,0.0,TBC,NaN
1783,51.63319: -0.195218,134617934,Available,"6 bedroom house for sale in Pine Grove, Totter...","An opportunity to acquire this spectacular, 6 ...",House,6.0,6.0,"£7,500,000",NaN,...,32 Totteridge Lane London N20 9QJ,,Totteridge & Whetstone Station:0.7136506677192...,FREEHOLD,NaN,Exempted,NaN,NaN,H,NaN


                    WORKING ON SCRAPED ZOOPLA DATA

In [336]:
#READING THE SCRAPED CSV FILE INTO A DATAFRAME CALLED df_zp
df_zp=pd.read_csv('results/zp_properties_all.csv')

#CLEANING UP THE SCRAPED DATA AND CREATING DESIRED COLUMNS

from bs4 import BeautifulSoup
#deleting irrelevant columns
df_zp = df_zp.drop(['photos', 'floor_plan','url', 'section'], axis=1)

#converting the coordinates to separate columns
import ast
df_zp['coordinates'] = df_zp['coordinates'].apply(ast.literal_eval)
df_zp['latitude'] = df_zp['coordinates'].apply(lambda x: x['lat'])
df_zp['longitude'] = df_zp['coordinates'].apply(lambda x: x['lng'])

#Extracting out and creating different columns from the agency column
df_zp['agency'] = df_zp['agency'].apply(ast.literal_eval)
df_zp['Agent_Name'] = df_zp['agency'].apply(lambda x: x['name'])
df_zp['Agent_Phone'] = df_zp['agency'].apply(lambda x: x['phone'])
df_zp['Agent_address'] = df_zp['agency'].apply(lambda x: x['address'])
df_zp['Agent_Post_code'] = df_zp['agency'].apply(lambda x: x['postcode'])

#Extracting out and creating different columns from the details column
df_zp['details'] = df_zp['details'].apply(ast.literal_eval)
df_zp['Bedrooms'] = df_zp['details'].apply(lambda x: x['numBeds'])
df_zp['Bathrooms'] = df_zp['details'].apply(lambda x: x['numBaths'])
df_zp['Receptions'] = df_zp['details'].apply(lambda x: x['numRecepts'])
df_zp['address'] = df_zp['details'].apply(lambda x: x['displayAddress'])
df_zp['address'] = df_zp['address'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df_zp['Area'] = df_zp['details'].apply(lambda x: x['areaName'])
df_zp['Outcode']=df_zp['details'].apply(lambda x: x['outcode'])
df_zp['Incode']=df_zp['details'].apply(lambda x: x['incode'])
df_zp['Region'] = df_zp['details'].apply(lambda x: x['regionName'])
df_zp['Tenure_type'] = df_zp['details'].apply(lambda x: x['tenure'])
df_zp['zindex'] = df_zp['details'].apply(lambda x: x['zindex'])
df_zp['Furnished_Status'] = df_zp['details'].apply(lambda x: x['furnishedState'])


# creating a postcode column by combining outcode and incode
df_zp['Postcode'] = df_zp['Outcode'].str.cat(df_zp['Incode'], sep=' ')

#Cleaning up the 'nearby' column by removing the dictionary keys and leaving the values
df_zp['nearby'] = df_zp['nearby'].apply(lambda x: ast.literal_eval(x))
df_zp['nearby'] = df_zp['nearby'].apply(lambda x: [f"{item['title']}:{item['distance']}" for item in x])
df_zp['nearby'] = df_zp['nearby'].apply(lambda x: ', '.join(x))

#cleaning up the description column by removing html tags
df_zp['description'] = df_zp['description'].str.replace('<strong>', '').str.replace('</strong>', '').str.replace('<br>', '')

#stripping off the list symbol from features column
df_zp['features'] = df_zp['features'].str.replace(r'\[|\]', '')

#dropping all the irrelevant columns
df_zp = df_zp.drop(['coordinates', 'agency','details','Outcode', 'Incode'], axis=1)

#dropping duplicates
df_zp.drop_duplicates(subset='id', keep='first', inplace=True)

#PREPROCESSING OF SCRAPED ZOOPLA DATA

#Prefixing all the columns of df_rm with ZP
df_zp = df_zp.add_prefix('ZP_')

#Creating a unique column called Coordinate
df_zp['Coordinate'] = df_zp['ZP_latitude'].astype(str) + ': ' + df_zp['ZP_longitude'].astype(str)

#making the coordinate column the first column
df_zp = df_zp[['Coordinate'] + df_zp.columns[:-1].tolist()]

#FINAL ZOOPLA DATAFRAME
df_zp

<ipython-input-336-3f6e6510b7a9>:51: FutureWarning: The default value of regex will change from True to False in a future version.
  df_zp['features'] = df_zp['features'].str.replace(r'\[|\]', '')


,Coordinate,ZP_id,ZP_title,ZP_description,ZP_price,ZP_type,ZP_date,ZP_category,ZP_features,ZP_nearby,...,ZP_Bedrooms,ZP_Bathrooms,ZP_Receptions,ZP_address,ZP_Area,ZP_Region,ZP_Tenure_type,ZP_zindex,ZP_Furnished_Status,ZP_Postcode
0,51.522098: -0.078848,46020849,2 bed flat for sale,Sandra Davidson are please to offer an opportu...,"£1,439,000",flat,2017-12-18T20:03:10,residential,"'Two Bedrooms', 'Open Plan Living/Kitchen area...","Shoreditch High Street:0.1, The Lyceum:0.2, Lo...",...,2.0,2.0,1.0,"2 Principal Place, Worship Street, London EC2A",London,London,leasehold,NaN,,EC2A 2FA
1,51.522098: -0.078848,46020848,1 bed flat for sale,Sandra Davidson are please to offer an opportu...,"£1,102,000",flat,2017-12-18T20:03:10,residential,"'Double Bedroom', 'Open Plan Living/Kitchen', ...","Shoreditch High Street:0.1, The Lyceum:0.2, Lo...",...,1.0,1.0,1.0,"2 Principal Place, Worship Street, London EC2A",London,London,leasehold,NaN,,EC2A 2FA
2,51.522098: -0.078848,46020845,1 bed flat for sale,Sandra Davidson are please to offer an opportu...,"£869,000",flat,2017-12-18T20:03:10,residential,"'Double Bedroom', 'Open Plan Living/Kitchen', ...","Shoreditch High Street:0.1, The Lyceum:0.2, Lo...",...,1.0,1.0,1.0,"2 Principal Place, Worship Street, London EC2A",London,London,leasehold,NaN,,EC2A 2FA
3,51.522098: -0.078848,46020846,2 bed flat for sale,Sandra Davidson are please to offer an opportu...,"£1,749,000",flat,2017-12-18T20:03:10,residential,"'Two Bedrooms', 'Open Plan Living/Kitchen area...","Shoreditch High Street:0.1, The Lyceum:0.2, Lo...",...,2.0,2.0,1.0,"2 Principal Place, Worship Street, London EC2A",London,London,leasehold,NaN,,EC2A 2FA
4,51.522098: -0.078848,46020867,2 bed flat for sale,Sandra Davidson are please to offer an opportu...,"£1,229,000",flat,2017-12-18T20:10:12,residential,"'Two Bedrooms', 'Open Plan Living/Kitchen area...","Shoreditch High Street:0.1, The Lyceum:0.2, Lo...",...,2.0,2.0,1.0,"2 Principal Place, Worship Street, London EC2A",London,London,leasehold,NaN,,EC2A 2FA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,51.537689: 0.12884,64704207,3 bed end terrace house for sale,Located close to Becontree train station and l...,"£425,000",end_terrace,2023-05-24T14:23:54,residential,,"The James Cambell Primary School:0.1, Becontre...",...,3.0,1.0,1.0,"Amesbury Road, Dagenham RM9",Dagenham,East of England,freehold,363713.0,,RM9 6AB
1021,51.544145: 0.093359,64705124,3 bed terraced house for sale,SummaryThree Bedroom Terrace House with Off St...,"£550,000",terraced,2023-05-24T15:15:58,residential,"'Three Bedroom House', 'Off Street Parking', '...","Eastbury Community School:0.2, Barking Abbey S...",...,3.0,1.0,2.0,"Ventnor Gardens, Barking IG11",Barking,East of England,freehold,515387.0,,IG11 9JY
1022,51.531291: 0.086555,64706751,2 bed property for sale,Sandra Davidson Estate Agents are pleased to p...,"£360,000",NaN,2023-05-24T17:31:56,residential,"'EPC tbc', 'Two bedrooms', 'Lounge', 'Kitchen'...","Gascoigne Primary School:0.2, Ripple Primary S...",...,2.0,1.0,1.0,"King Edwards Road, Barking IG11",Barking,East of England,None,301597.0,,IG11 7TP
1023,51.536573: 0.095925,64705995,3 bed end terrace house for sale,SummaryThree Bedroom Recently Refurbished End ...,"£400,000",end_terrace,2023-05-24T16:15:18,residential,"'Three Bedroom House', 'End of Terrace', 'Off ...","Eastbury Primary School:0.2, Ripple Primary Sc...",...,3.0,1.0,1.0,"Lambourne Road, Barking IG11",Barking,East of England,freehold,341860.0,,IG11 9PR


In [338]:
#Merging both dataframes on common column which is 'Coordinate'
df_combined = pd.merge(df_rm, df_zp, on='Coordinate', how='outer')

In [339]:
#displaying the final combined dataset
df_combined

,Coordinate,RM_id,RM_Status,RM_title,RM_description,RM_property_type,RM_bedrooms,RM_bathrooms,RM_price,RM_price_sqft,...,ZP_Bedrooms,ZP_Bathrooms,ZP_Receptions,ZP_address,ZP_Area,ZP_Region,ZP_Tenure_type,ZP_zindex,ZP_Furnished_Status,ZP_Postcode
0,51.5166: -0.080557,74504326.0,Available,1 bedroom apartment for sale in One Bishopsgat...,"This 21st floor, one bedroom apartment is well...",Apartment,1.0,1.0,"£1,325,000",£2086.61 per sq. ft.,...,1.0,1.0,1.0,"One Bishopsgate Plaza, 80 Houndsditch, London ...",,London,leasehold,NaN,,EC3A 7AB
1,51.514636: -0.109152,133932536.0,Available,"Studio apartment for sale in Crane Court, Lond...",Recently refurbished studio in Central London....,Apartment,NaN,1.0,"£450,000",NaN,...,NaN,1.0,1.0,"Crane Court, London EC4A",London,London,share_of_freehold,NaN,,EC4A 2EJ
2,51.519342: -0.089223,128173136.0,Available,"Studio flat for sale in The Heron, Moorgate, L...","Set on the 14th floor, this immaculate studio ...",Studio,NaN,NaN,"£575,000",£1399.03 per sq. ft.,...,2.0,2.0,1.0,"Moor Lane, Moorgate, London EC2Y",London,London,leasehold,1181758.0,,EC2Y 9AP
3,51.519342: -0.089223,128173136.0,Available,"Studio flat for sale in The Heron, Moorgate, L...","Set on the 14th floor, this immaculate studio ...",Studio,NaN,NaN,"£575,000",£1399.03 per sq. ft.,...,NaN,NaN,NaN,"The Heron, Moorgate, London EC2Y",London,London,leasehold,1181758.0,,EC2Y 9AP
4,51.519342: -0.089223,82267581.0,Available,"2 bedroom flat for sale in Moor Lane, Moorgate...",This impressive 2 bedroom apartment boasts imm...,Flat,2.0,2.0,"£1,400,000",£1536.77 per sq. ft.,...,2.0,2.0,1.0,"Moor Lane, Moorgate, London EC2Y",London,London,leasehold,1181758.0,,EC2Y 9AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774,51.534584: 0.079791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,NaN,"Cooke Street, Barking IG11",Barking,East of England,leasehold,303211.0,,IG11 7AF
2775,51.537689: 0.12884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,"Amesbury Road, Dagenham RM9",Dagenham,East of England,freehold,363713.0,,RM9 6AB
2776,51.544145: 0.093359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,2.0,"Ventnor Gardens, Barking IG11",Barking,East of England,freehold,515387.0,,IG11 9JY
2777,51.536573: 0.095925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,"Lambourne Road, Barking IG11",Barking,East of England,freehold,341860.0,,IG11 9PR


In [340]:
# Reordering the columns
column_order = ['Coordinate','RM_id', 'ZP_id', 'RM_Status', 'RM_Listinghistory', 'ZP_date', 'RM_title', 'ZP_title', 'RM_price', 'ZP_price', 'RM_price_sqft', 'ZP_category', 'RM_property_type', 'ZP_type', 'RM_description', 'ZP_description', 'RM_features', 'ZP_features', 'RM_bedrooms', 'ZP_Bedrooms', 'RM_bathrooms', 'ZP_Bathrooms', 'ZP_Receptions', 'RM_photos', 'RM_floorplans', 'RM_nearest_stations', 'ZP_nearby', 'RM_nearest_airports', 'RM_latitude', 'ZP_latitude', 'RM_longitude', 'ZP_longitude', 'RM_Tenure_type', 'ZP_Tenure_type', 'RM_Lease_year_left', 'RM_CouncilTax_Status', 'RM_annualGroundRent', 'RM_annualServiceCharge', 'RM_councilTaxBand', 'RM_domesticRates', 'RM_address', 'ZP_address', 'ZP_Area', 'ZP_Region', 'ZP_Postcode', 'ZP_zindex', 'RM_Agent_Name', 'ZP_Agent_Name', 'RM_Agent_No', 'ZP_Agent_Phone', 'RM_Agent_Address', 'ZP_Agent_address', 'ZP_Agent_Post_code', 'ZP_Furnished_Status']

# Reorder the columns
df_combined = df_combined[column_order]

In [341]:
df_combined

,Coordinate,RM_id,ZP_id,RM_Status,RM_Listinghistory,ZP_date,RM_title,ZP_title,RM_price,ZP_price,...,ZP_Postcode,ZP_zindex,RM_Agent_Name,ZP_Agent_Name,RM_Agent_No,ZP_Agent_Phone,RM_Agent_Address,ZP_Agent_address,ZP_Agent_Post_code,ZP_Furnished_Status
0,51.5166: -0.080557,74504326.0,61273901.0,Available,Added on 03/10/2019,2019-10-03T16:03:59,1 bedroom apartment for sale in One Bishopsgat...,1 bed flat for sale,"£1,325,000","£1,325,000",...,EC3A 7AB,NaN,Savills,Savills - Margaret Street New Homes,020 3834 7939,020 8022 3382,33 Margaret Street London W1G 0JD,"33 Margaret Street, London",W1G 0JD,
1,51.514636: -0.109152,133932536.0,64455464.0,Available,Added on 21/04/2023,2023-04-21T12:44:44,"Studio apartment for sale in Crane Court, Lond...",Studio for sale,"£450,000","£450,000",...,EC4A 2EJ,NaN,Savills,Savills - Clerkenwell,020 3910 6023,020 8022 3157,"122-124 St Johns Street, London, EC1V 4JS","122/124 St John Street, Clerkenwell",EC1V 4JS,
2,51.519342: -0.089223,128173136.0,60071293.0,Available,Reduced on 09/11/2022,2021-10-28T18:06:32,"Studio flat for sale in The Heron, Moorgate, L...",2 bed flat for sale,"£575,000","£1,400,000",...,EC2Y 9AP,1181758.0,Foxtons,Foxtons - Clerkenwell,020 3834 7972,020 3463 2552,"50 - 54 Clerkenwell Road, London, EC1M 5PS","50-54 Clerkenwell Road, London",EC1M 5PS,
3,51.519342: -0.089223,128173136.0,62679037.0,Available,Reduced on 09/11/2022,2022-10-18T18:05:44,"Studio flat for sale in The Heron, Moorgate, L...",Studio for sale,"£575,000","£575,000",...,EC2Y 9AP,1181758.0,Foxtons,Foxtons - Clerkenwell,020 3834 7972,020 3463 2552,"50 - 54 Clerkenwell Road, London, EC1M 5PS","50-54 Clerkenwell Road, London",EC1M 5PS,
4,51.519342: -0.089223,82267581.0,60071293.0,Available,Added on 28/10/2021,2021-10-28T18:06:32,"2 bedroom flat for sale in Moor Lane, Moorgate...",2 bed flat for sale,"£1,400,000","£1,400,000",...,EC2Y 9AP,1181758.0,Foxtons,Foxtons - Clerkenwell,020 3834 7972,020 3463 2552,"50 - 54 Clerkenwell Road, London, EC1M 5PS","50-54 Clerkenwell Road, London",EC1M 5PS,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774,51.534584: 0.079791,NaN,64702813.0,NaN,NaN,2023-05-24T12:59:37,NaN,2 bed flat for sale,NaN,"£315,000",...,IG11 7AF,303211.0,NaN,Just Move Ltd,NaN,020 8166 3233,NaN,"Unit 6 Balfour House, 390-398, High Road, Ilfo...",IG1 1TL,
2775,51.537689: 0.12884,NaN,64704207.0,NaN,NaN,2023-05-24T14:23:54,NaN,3 bed end terrace house for sale,NaN,"£425,000",...,RM9 6AB,363713.0,NaN,Bairstow Eves - Dagenham,NaN,020 3463 0261,NaN,"252 Heathway, Dagenham",RM10 8QS,
2776,51.544145: 0.093359,NaN,64705124.0,NaN,NaN,2023-05-24T15:15:58,NaN,3 bed terraced house for sale,NaN,"£550,000",...,IG11 9JY,515387.0,NaN,William H Brown Incorporating Porter Glenny - ...,NaN,020 8033 8686,NaN,"3 Faircross Parade, Longbridge Road, Barking",IG11 8UN,
2777,51.536573: 0.095925,NaN,64705995.0,NaN,NaN,2023-05-24T16:15:18,NaN,3 bed end terrace house for sale,NaN,"£400,000",...,IG11 9PR,341860.0,NaN,William H Brown Incorporating Porter Glenny - ...,NaN,020 8033 8686,NaN,"3 Faircross Parade, Longbridge Road, Barking",IG11 8UN,


In [343]:
#creating new columns with value from either rightmove or zoopla data
df_combined['Listed_date'] = df_combined['ZP_date'].fillna(df_combined['RM_Listinghistory'])

df_combined.loc[~df_combined['RM_id'].isna(), 'Status'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_Status']
df_combined['Title'] = df_combined['RM_title'].fillna(df_combined['ZP_title'])
df_combined['Price'] = df_combined['RM_price'].fillna(df_combined['ZP_price'])
df_combined.loc[~df_combined['RM_id'].isna(), 'Price_sqft'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_price_sqft']
df_combined.loc[~df_combined['ZP_id'].isna(), 'Category'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_category']

df_combined['Property_type'] = df_combined['RM_property_type'].fillna(df_combined['ZP_type'])
df_combined['Description'] = df_combined['RM_description'].fillna(df_combined['ZP_description'])
df_combined['Features'] = df_combined['RM_features'].fillna(df_combined['ZP_features'])
df_combined['Bedrooms'] = df_combined['RM_bedrooms'].fillna(df_combined['ZP_Bedrooms'])
df_combined['Bathrooms'] = df_combined['RM_bathrooms'].fillna(df_combined['ZP_Bathrooms'])
df_combined.loc[~df_combined['ZP_id'].isna(), 'Receptions'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_Receptions']
df_combined.loc[~df_combined['RM_id'].isna(), 'Photos'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_photos']
df_combined.loc[~df_combined['RM_id'].isna(), 'Floorplans'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_floorplans']
df_combined.loc[~df_combined['RM_id'].isna(), 'Nearest_stations'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_nearest_stations']
df_combined.loc[~df_combined['ZP_id'].isna(), 'Nearby_landmarks'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_nearby']

df_combined.loc[~df_combined['RM_id'].isna(), 'Nearest_airports'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_nearest_airports']

df_combined['Latitude'] = df_combined['RM_latitude'].fillna(df_combined['ZP_longitude'])
df_combined['Longitude'] = df_combined['RM_longitude'].fillna(df_combined['ZP_Tenure_type'])
df_combined['Tenure_type'] = df_combined['RM_Tenure_type'].fillna(df_combined['ZP_title'])
df_combined.loc[~df_combined['RM_id'].isna(), 'Lease_year_left'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_Lease_year_left']
df_combined.loc[~df_combined['RM_id'].isna(), 'CouncilTax_Status'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_CouncilTax_Status']
df_combined.loc[~df_combined['RM_id'].isna(), 'Annual_Ground_Rent'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_annualGroundRent']
df_combined.loc[~df_combined['RM_id'].isna(), 'Annual_Service_Charge'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_annualServiceCharge']
df_combined.loc[~df_combined['RM_id'].isna(), 'Council_Tax_Band'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_councilTaxBand']
df_combined.loc[~df_combined['RM_id'].isna(), 'Domestic_Rates'] = df_combined.loc[~df_combined['RM_id'].isna(), 'RM_domesticRates']

df_combined['Address'] = df_combined['ZP_address'].fillna(df_combined['RM_address'])
df_combined.loc[~df_combined['ZP_id'].isna(), 'Area'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_Area']
df_combined.loc[~df_combined['ZP_id'].isna(), 'Region'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_Region']
df_combined.loc[~df_combined['ZP_id'].isna(), 'Postcode'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_Postcode']
df_combined.loc[~df_combined['ZP_id'].isna(), 'Zindex'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_zindex']

df_combined['Agent_name'] = df_combined['RM_Agent_Name'].fillna(df_combined['ZP_Agent_Name'])
df_combined['Agent_Phone_no'] = df_combined['RM_Agent_No'].fillna(df_combined['ZP_Agent_Phone'])
df_combined['Agent_address'] = df_combined['RM_Agent_Address'].fillna(df_combined['ZP_Agent_address'])
df_combined.loc[~df_combined['ZP_id'].isna(), 'Agent_post_code'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_Agent_Post_code']
df_combined.loc[~df_combined['ZP_id'].isna(), 'Furnished_status'] = df_combined.loc[~df_combined['ZP_id'].isna(), 'ZP_Furnished_Status']




<ipython-input-343-2cd498fa772c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined['Listed_date'] = df_combined['ZP_date'].fillna(df_combined['RM_Listinghistory'])
C:\Users\JUDE\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\JUDE\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [344]:
df_combined

,Coordinate,RM_id,ZP_id,RM_Status,RM_Listinghistory,ZP_date,RM_title,ZP_title,RM_price,ZP_price,...,Address,Area,Region,Postcode,Zindex,Agent_name,Agent_Phone_no,Agent_address,Agent_post_code,Furnished_status
0,51.5166: -0.080557,74504326.0,61273901.0,Available,Added on 03/10/2019,2019-10-03T16:03:59,1 bedroom apartment for sale in One Bishopsgat...,1 bed flat for sale,"£1,325,000","£1,325,000",...,"One Bishopsgate Plaza, 80 Houndsditch, London ...",,London,EC3A 7AB,NaN,Savills,020 3834 7939,33 Margaret Street London W1G 0JD,W1G 0JD,
1,51.514636: -0.109152,133932536.0,64455464.0,Available,Added on 21/04/2023,2023-04-21T12:44:44,"Studio apartment for sale in Crane Court, Lond...",Studio for sale,"£450,000","£450,000",...,"Crane Court, London EC4A",London,London,EC4A 2EJ,NaN,Savills,020 3910 6023,"122-124 St Johns Street, London, EC1V 4JS",EC1V 4JS,
2,51.519342: -0.089223,128173136.0,60071293.0,Available,Reduced on 09/11/2022,2021-10-28T18:06:32,"Studio flat for sale in The Heron, Moorgate, L...",2 bed flat for sale,"£575,000","£1,400,000",...,"Moor Lane, Moorgate, London EC2Y",London,London,EC2Y 9AP,1181758.0,Foxtons,020 3834 7972,"50 - 54 Clerkenwell Road, London, EC1M 5PS",EC1M 5PS,
3,51.519342: -0.089223,128173136.0,62679037.0,Available,Reduced on 09/11/2022,2022-10-18T18:05:44,"Studio flat for sale in The Heron, Moorgate, L...",Studio for sale,"£575,000","£575,000",...,"The Heron, Moorgate, London EC2Y",London,London,EC2Y 9AP,1181758.0,Foxtons,020 3834 7972,"50 - 54 Clerkenwell Road, London, EC1M 5PS",EC1M 5PS,
4,51.519342: -0.089223,82267581.0,60071293.0,Available,Added on 28/10/2021,2021-10-28T18:06:32,"2 bedroom flat for sale in Moor Lane, Moorgate...",2 bed flat for sale,"£1,400,000","£1,400,000",...,"Moor Lane, Moorgate, London EC2Y",London,London,EC2Y 9AP,1181758.0,Foxtons,020 3834 7972,"50 - 54 Clerkenwell Road, London, EC1M 5PS",EC1M 5PS,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774,51.534584: 0.079791,NaN,64702813.0,NaN,NaN,2023-05-24T12:59:37,NaN,2 bed flat for sale,NaN,"£315,000",...,"Cooke Street, Barking IG11",Barking,East of England,IG11 7AF,303211.0,Just Move Ltd,020 8166 3233,"Unit 6 Balfour House, 390-398, High Road, Ilfo...",IG1 1TL,
2775,51.537689: 0.12884,NaN,64704207.0,NaN,NaN,2023-05-24T14:23:54,NaN,3 bed end terrace house for sale,NaN,"£425,000",...,"Amesbury Road, Dagenham RM9",Dagenham,East of England,RM9 6AB,363713.0,Bairstow Eves - Dagenham,020 3463 0261,"252 Heathway, Dagenham",RM10 8QS,
2776,51.544145: 0.093359,NaN,64705124.0,NaN,NaN,2023-05-24T15:15:58,NaN,3 bed terraced house for sale,NaN,"£550,000",...,"Ventnor Gardens, Barking IG11",Barking,East of England,IG11 9JY,515387.0,William H Brown Incorporating Porter Glenny - ...,020 8033 8686,"3 Faircross Parade, Longbridge Road, Barking",IG11 8UN,
2777,51.536573: 0.095925,NaN,64705995.0,NaN,NaN,2023-05-24T16:15:18,NaN,3 bed end terrace house for sale,NaN,"£400,000",...,"Lambourne Road, Barking IG11",Barking,East of England,IG11 9PR,341860.0,William H Brown Incorporating Porter Glenny - ...,020 8033 8686,"3 Faircross Parade, Longbridge Road, Barking",IG11 8UN,


In [347]:
#We can now drop used individual columns and some unuseful columns

df_combined_final = df_combined.drop(['Coordinate','RM_id','ZP_id','RM_Status','RM_Listinghistory', 'ZP_date', 'RM_title', 'ZP_title', 'RM_price', 'ZP_price', 'RM_price_sqft', 'ZP_category', 'RM_property_type', 'ZP_type', 'RM_description', 'ZP_description', 'RM_features', 'ZP_features', 'RM_bedrooms', 'ZP_Bedrooms', 'RM_bathrooms', 'ZP_Bathrooms', 'ZP_Receptions', 'RM_photos', 'RM_floorplans', 'RM_nearest_stations', 'ZP_nearby', 'RM_nearest_airports', 'RM_latitude', 'ZP_latitude', 'RM_longitude', 'ZP_longitude', 'RM_Tenure_type', 'ZP_Tenure_type', 'RM_Lease_year_left', 'RM_CouncilTax_Status', 'RM_annualGroundRent', 'RM_annualServiceCharge', 'RM_councilTaxBand', 'RM_domesticRates', 'RM_address', 'ZP_address', 'ZP_Area', 'ZP_Region', 'ZP_Postcode', 'ZP_zindex', 'RM_Agent_Name', 'ZP_Agent_Name', 'RM_Agent_No', 'ZP_Agent_Phone', 'RM_Agent_Address', 'ZP_Agent_address', 'ZP_Agent_Post_code', 'ZP_Furnished_Status','Furnished_status','Zindex'], axis=1)
#droping the pound sign from price and price per sq
df_combined_final['Price'] = df_combined_final['Price'].str.replace('£', '')
df_combined_final['Price_sqft'] = df_combined_final['Price_sqft'].str.replace('£', '').str.replace('per sq. ft.', '')



<ipython-input-347-b3f4444c555e>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combined_final['Price_sqft'] = df_combined_final['Price_sqft'].str.replace('£', '').str.replace('per sq. ft.', '')


,Listed_date,Status,Title,Price,Price_sqft,Category,Property_type,Description,Features,Bedrooms,...,Council_Tax_Band,Domestic_Rates,Address,Area,Region,Postcode,Agent_name,Agent_Phone_no,Agent_address,Agent_post_code
0,2019-10-03T16:03:59,Available,1 bedroom apartment for sale in One Bishopsgat...,"1,325,000",2086.61,residential,Apartment,"This 21st floor, one bedroom apartment is well...","""Unique views of some of London's most iconic ...",1.0,...,H,NaN,"One Bishopsgate Plaza, 80 Houndsditch, London ...",,London,EC3A 7AB,Savills,020 3834 7939,33 Margaret Street London W1G 0JD,W1G 0JD
1,2023-04-21T12:44:44,Available,"Studio apartment for sale in Crane Court, Lond...","450,000",NaN,residential,Apartment,Recently refurbished studio in Central London....,"'Recently Refurbished', 'Centrally Located', '...",NaN,...,C,NaN,"Crane Court, London EC4A",London,London,EC4A 2EJ,Savills,020 3910 6023,"122-124 St Johns Street, London, EC1V 4JS",EC1V 4JS
2,2021-10-28T18:06:32,Available,"Studio flat for sale in The Heron, Moorgate, L...","575,000",1399.03,residential,Studio,"Set on the 14th floor, this immaculate studio ...","'OFFERS IN EXCESS OF', 'Gorgeous 14th Floor ap...",2.0,...,NaN,NaN,"Moor Lane, Moorgate, London EC2Y",London,London,EC2Y 9AP,Foxtons,020 3834 7972,"50 - 54 Clerkenwell Road, London, EC1M 5PS",EC1M 5PS
3,2022-10-18T18:05:44,Available,"Studio flat for sale in The Heron, Moorgate, L...","575,000",1399.03,residential,Studio,"Set on the 14th floor, this immaculate studio ...","'OFFERS IN EXCESS OF', 'Gorgeous 14th Floor ap...",NaN,...,NaN,NaN,"The Heron, Moorgate, London EC2Y",London,London,EC2Y 9AP,Foxtons,020 3834 7972,"50 - 54 Clerkenwell Road, London, EC1M 5PS",EC1M 5PS
4,2021-10-28T18:06:32,Available,"2 bedroom flat for sale in Moor Lane, Moorgate...","1,400,000",1536.77,residential,Flat,This impressive 2 bedroom apartment boasts imm...,'High-specification apartment set on the 8th f...,2.0,...,NaN,NaN,"Moor Lane, Moorgate, London EC2Y",London,London,EC2Y 9AP,Foxtons,020 3834 7972,"50 - 54 Clerkenwell Road, London, EC1M 5PS",EC1M 5PS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774,2023-05-24T12:59:37,NaN,2 bed flat for sale,"315,000",NaN,residential,flat,Just Move is pleased to present you the opport...,"'Communal garden', 'Off street parking', 'Cent...",2.0,...,NaN,NaN,"Cooke Street, Barking IG11",Barking,East of England,IG11 7AF,Just Move Ltd,020 8166 3233,"Unit 6 Balfour House, 390-398, High Road, Ilfo...",IG1 1TL
2775,2023-05-24T14:23:54,NaN,3 bed end terrace house for sale,"425,000",NaN,residential,end_terrace,Located close to Becontree train station and l...,,3.0,...,NaN,NaN,"Amesbury Road, Dagenham RM9",Dagenham,East of England,RM9 6AB,Bairstow Eves - Dagenham,020 3463 0261,"252 Heathway, Dagenham",RM10 8QS
2776,2023-05-24T15:15:58,NaN,3 bed terraced house for sale,"550,000",NaN,residential,terraced,SummaryThree Bedroom Terrace House with Off St...,"'Three Bedroom House', 'Off Street Parking', '...",3.0,...,NaN,NaN,"Ventnor Gardens, Barking IG11",Barking,East of England,IG11 9JY,William H Brown Incorporating Porter Glenny - ...,020 8033 8686,"3 Faircross Parade, Longbridge Road, Barking",IG11 8UN
2777,2023-05-24T16:15:18,NaN,3 bed end terrace house for sale,"400,000",NaN,residential,end_terrace,SummaryThree Bedroom Recently Refurbished End ...,"'Three Bedroom House', 'End of Terrace', 'Off ...",3.0,...,NaN,NaN,"Lambourne Road, Barking IG11",Barking,East of England,IG11 9PR,William H Brown Incorporating Porter Glenny - ...,020 8033 8686,"3 Faircross Parade, Longbridge Road, Barking",IG11 8UN


In [ ]:
#THE FINAL DATA FRAME
df_combined_final

In [349]:
#Coverting the final dataframe to a csv file called final_dataset
df_combined_final.to_csv('results/final_dataset.csv', index=False)